In [1]:
import numpy as np
import os 
import pandas as pd
import cv2
import torch
import matplotlib.pyplot as plt
from ipywidgets import interact
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torchvision
from torch import nn
import torchsummary
from torch.utils.data import DataLoader
from collections import defaultdict
from torchvision.utils import make_grid

import torch
# from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from transformers import BertTokenizer, BertModel, BertForMaskedLM
import logging
import matplotlib.pyplot as plt

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

## Image Encoder

In [6]:
class Image_Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        
        resnet = torchvision.models.resnet34(weights = 'IMAGENET1K_V1')
        layers = [m for m in resnet.children()]
        
#         self.backbone = nn.Sequential(*layers[:-2]) 
        self.backbone = resnet
        self.head = nn.Sequential(
                nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1, padding=0,bias=False),
                nn.BatchNorm2d(256),
                nn.ReLU(inplace=True),
                nn.Conv2d(in_channels=256, out_channels=128, kernel_size=3, padding=1,bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.Conv2d(in_channels=128, out_channels=32, kernel_size=3, padding=1,bias=False),
                nn.BatchNorm2d(32),
                nn.ReLU(inplace=True),
                nn.Conv2d(in_channels=32, out_channels=4, kernel_size=3, padding=1,bias=False),
                nn.BatchNorm2d(4),
                nn.ReLU(inplace=True),
                nn.Flatten(),
                nn.Linear(in_features=784, out_features=768)
            
            )
        
#         self.head = nn.Sequential(
#                 nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1, padding=0,bias=False),
#                 nn.BatchNorm2d(256),
#                 nn.ReLU(inplace=True),
#                 nn.Conv2d(in_channels=256, out_channels=128, kernel_size=3, padding=1,bias=False),
#                 nn.BatchNorm2d(128),
#                 nn.ReLU(inplace=True),
#                 nn.Conv2d(in_channels=128, out_channels=32, kernel_size=3, padding=1,bias=False),
#                 nn.BatchNorm2d(32),
#                 nn.ReLU(inplace=True),
#                 nn.Conv2d(in_channels=32, out_channels=1, kernel_size=3, padding=1,bias=False),
#                 nn.BatchNorm2d(1),
#                 nn.ReLU(inplace=True),
#                 nn.Flatten()
#             )
    def forward(self, x):
        out = self.backbone(x)
#         out = self.head(out) # final output=> (1, 196)
        return out

In [7]:
Image_Enc = Image_Encoder()
Image_Enc.to(device)
torchsummary.summary(Image_Enc, (3,448,448))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           9,408
       BatchNorm2d-2         [-1, 64, 224, 224]             128
              ReLU-3         [-1, 64, 224, 224]               0
         MaxPool2d-4         [-1, 64, 112, 112]               0
            Conv2d-5         [-1, 64, 112, 112]          36,864
       BatchNorm2d-6         [-1, 64, 112, 112]             128
              ReLU-7         [-1, 64, 112, 112]               0
            Conv2d-8         [-1, 64, 112, 112]          36,864
       BatchNorm2d-9         [-1, 64, 112, 112]             128
             ReLU-10         [-1, 64, 112, 112]               0
       BasicBlock-11         [-1, 64, 112, 112]               0
           Conv2d-12         [-1, 64, 112, 112]          36,864
      BatchNorm2d-13         [-1, 64, 112, 112]             128
             ReLU-14         [-1, 64, 1